In [1]:
import json
import pandas as pd
from src.umls import Umls
umls = Umls('umls/processed')
mrconso = pd.read_feather('umls/processed/mrconso.feather')

In [2]:
def get_name(cui):
    try:
        return mrconso[(mrconso.CUI==cui)&(mrconso.ISPREF=='Y')].values[0,1]
    except:
        print("Failed getting",cui)
        return ""
get_name('C0000039')

'Dipalmitoylphosphatidylcholine (substance)'

In [3]:
with open('results/bb_log_ce.json') as f:
    result = json.load(f)['result']

results = []
for r in result:
    mention = r['mention']
    gold_cui = r['golden_cui']
    candidates = r['candidates']

    try:
        first_correct = [c['label'] for c in candidates].index(1)
    except:
        first_correct = -1
    pred = candidates[0]['name']
    pred_cui = candidates[0]['cui']
    pred_dist = umls.dist(gold_cui,pred_cui)
    correct = get_name(gold_cui)
    results.append((mention, first_correct, pred_dist, correct, pred, gold_cui, pred_cui))
df = pd.DataFrame(results, columns=['mention','first_correct','pred_dist','correct','pred','gold_cui','pred_cui'])

Failed getting C0178638
Failed getting C1971825
Failed getting C0178638
Failed getting C0162712
Failed getting C0005001
Failed getting C0085934
Failed getting C0064238
Failed getting C0699055
Failed getting C0162712
Failed getting C1456822
Failed getting C0699194
Failed getting C0182440
Failed getting C0591585
Failed getting C0733815
Failed getting C0064956
Failed getting C0728762
Failed getting C0592168
Failed getting C0699187
Failed getting C0086787
Failed getting C0067044
Failed getting C0728763
Failed getting C0086787
Failed getting C0719509
Failed getting C0699129
Failed getting C0593507
Failed getting C0086787
Failed getting C0728763
Failed getting C0086787
Failed getting C0728762
Failed getting C0178638
Failed getting C0699129
Failed getting C0699129
Failed getting C0086787
Failed getting C0220892
Failed getting C0728762
Failed getting C0086787
Failed getting C0220892
Failed getting C0699034
Failed getting C0728762
Failed getting C0592168
Failed getting C0086787
Failed getting C

In [34]:
errors.loc[(errors.error_type=='')&(errors.mention==errors.pred),'error_type'] = 'exact match'
errors

,mention,first_correct,pred_dist,correct,pred,gold_cui,pred_cui,error_type
6,difficulty,1,6,Difficult,difficulty,C0332218,C1299586,exact match
8,heart rate,-1,1,Pulse taking (procedure),heart rate monitoring,C0034107,C0199637,
10,illappearing,-1,4,Looks ill,feeling empty,C0459686,C0557912,
18,injury to his eyes,4,1,Periocular injury,"injury of eye, nos",C0560619,C0015408,
19,airway observation,-1,5,Functional finding of respiratory tract (finding),oral airway observation,C0577917,C0457430,
...,...,...,...,...,...,...,...,...
5286,detrusor,3,14,Detrusor muscle of urinary bladder,detrusor pressure,C1288327,C0429766,
5293,nephrostomy,1,1,Insertion of nephrostomy,nephrostomy,C0278314,C2937272,exact match
5295,electrolyte,-1,6,Electrolyte levels - finding,electrolytes measurement,C0428284,C0201992,
5298,history of present illness,-1,8,History of present illness section,history of present illness,C1827596,C5399914,exact match


In [48]:
# umls = Umls('umls/processed')
print(len(df))
errors = df[df.first_correct!=0]
errors.insert(7,'error_type',['']*len(errors))
errors.loc[(errors.error_type=='')&(errors.mention==errors.pred),'error_type'] = 'exact match'
errors.loc[(errors.error_type=='')&(errors.pred_dist==1), 'error_type'] = 'parent/child'
print(len(errors), len(errors[errors.error_type=='']))
errors.head(5)

5302
941 676


,mention,first_correct,pred_dist,correct,pred,gold_cui,pred_cui,error_type
6,difficulty,1,6,Difficult,difficulty,C0332218,C1299586,exact match
8,heart rate,-1,1,Pulse taking (procedure),heart rate monitoring,C0034107,C0199637,parent/child
10,illappearing,-1,4,Looks ill,feeling empty,C0459686,C0557912,
18,injury to his eyes,4,1,Periocular injury,"injury of eye, nos",C0560619,C0015408,parent/child
19,airway observation,-1,5,Functional finding of respiratory tract (finding),oral airway observation,C0577917,C0457430,


In [52]:
# errors[errors.error_type==''].to_csv('errors.csv')
errors[errors.error_type=='exact match'].head(50)

,mention,first_correct,pred_dist,correct,pred,gold_cui,pred_cui,error_type
6,difficulty,1,6,Difficult,difficulty,C0332218,C1299586,exact match
44,msir,-1,14,Morphine sulfate,msir,C0066814,C0700789,exact match
48,application,1,6,Application (procedure),application,C0185125,C1533134,exact match
60,hydration,-1,10,Fluid management,hydration,C0553741,C1321013,exact match
61,application,1,6,Application (procedure),application,C0185125,C1533134,exact match
67,application,1,6,Application (procedure),application,C0185125,C1533134,exact match
201,worsening,1,3,Worsening,worsening,C0332271,C1457868,exact match
204,flat,1,10,Flat,flat,C0205324,C0457933,exact match
221,transverse mesocolon,1,1,Transverse mesocolon,transverse mesocolon,C0230254,C1280340,exact match
231,transverse mesocolon,1,1,Transverse mesocolon,transverse mesocolon,C0230254,C1280340,exact match


In [43]:
mrconso[mrconso.CUI=='C1971825']

,CUI,STR,ISPREF
1416087,C1971825,Peri-Colace Reformulated Feb 2008,N


In [5]:
cui = 'C0741494'
# cui = 'C0201913'
umls.parents[cui]
display(mrconso[(mrconso.CUI==cui)&(mrconso.ISPREF=='Y')])
umls.dist(cui,'C0201913')

,CUI,STR,ISPREF
990505,C0741494,Elevated total bilirubin (finding),Y


Common ancestors: {'C1285556', 'C2720507'}
[{'C0741494'}, {'C0311468'}, {'C0476449', 'C0429621', 'C1287366', 'C1287365', 'C0586731', 'C2711264'}, {'C0560218', 'C5191618', 'C0428132', 'C0428212', 'C1287364', 'C0037088', 'C0586721', 'C0574125', 'C1285556', 'C0159132', 'C2711908', 'C0427350', 'C4316334', 'C1290915', 'C1287340', 'C0574124'}, {'C1290907', 'C1298232', 'C0587081', 'C1261153', 'C2711906', 'C0587096', 'C0476445', 'C0243095', 'C2317510', 'C1287327', 'C2720507'}]
[{'C0201913'}, {'C0344395', 'C1266246'}, {'C0022885', 'C0428428', 'C0430027', 'C2316799', 'C0201682'}, {'C1293083', 'C1293916', 'C0184661', 'C1261322', 'C0242485', 'C0420583'}, {'C1292787', 'C0449851', 'C1292784', 'C0039593', 'C0430022', 'C0420206', 'C0557985', 'C0441472', 'C1292786', 'C1285556', 'C2720507'}]


7

In [9]:
cui = 'C0344395'
display(mrconso[(mrconso.CUI==cui)&(mrconso.ISPREF=='Y')])

,CUI,STR,ISPREF
488810,C0344395,Bilirubin level,Y
488811,C0344395,Bilirubin measurement (procedure),Y


In [106]:
cui = 'C0022671'
display(mrconso[(mrconso.CUI==cui)&(mrconso.ISPREF=='Y')])
umls.dist('C0032821','C0202194')
p = [c for n,c in umls.parents[cui]]
print("Parents",p)
display(mrconso[(mrconso.CUI.isin(p))&(mrconso.ISPREF=='Y')])
k = [c for n,c in umls.children[cui]]
print("Children", k)
display(mrconso[(mrconso.CUI.isin(k))&(mrconso.ISPREF=='Y')])

,CUI,STR,ISPREF
55752,C0022671,Kidney transplantation,Y
55754,C0022671,Transplantation of kidney,Y
55755,C0022671,Transplantation of kidney NOS,Y
55756,C0022671,Transplant of kidney,Y
55758,C0022671,Kidney transplant,Y
55761,C0022671,Renal transplant,Y
55763,C0022671,Renal graft,Y
55764,C0022671,Tx - Kidney transplantation,Y
55765,C0022671,Tx - Renal transplantation,Y
55766,C0022671,Transplant of kidney (procedure),Y


Parents ['C1285344', 'C1293609', 'C0198582', 'C0194053', 'C1293222', 'C0198788', 'C1293614', 'C1302112', 'C0730400']


,CUI,STR,ISPREF
202590,C0194053,Kidney operation,Y
202592,C0194053,Kidney operations NOS,Y
202594,C0194053,Kidney operations,Y
202596,C0194053,Operation on kidney,Y
202598,C0194053,Renal operation,Y
202601,C0194053,Renal operations,Y
202602,C0194053,Kidney operation (procedure),Y
202603,C0194053,Kidney operations NOS (procedure),Y
202604,C0194053,Operation on kidney (procedure),Y
217021,C0198582,Transplantation of abdominal tissue,Y


Children ['C0194194', 'C0401175', 'C0730392', 'C4305325', 'C4707445', 'C4707446', 'C5190484']


,CUI,STR,ISPREF
203055,C0194194,Autotransplantation of kidney,Y
203057,C0194194,Renal autotransplantation,Y
203059,C0194194,Renal autotransplantation or reimplantation of...,Y
203061,C0194194,Autotransplant kidney,Y
203063,C0194194,Autotransplantation of kidney (procedure),Y
203064,C0194194,Renal autotransplantation (procedure),Y
549983,C0401175,Other specified transplantation of kidney,Y
549984,C0401175,Other specified transplantation of kidney (pro...,Y
986782,C0730392,Donor renal transplantation,Y
986783,C0730392,Donor renal transplantation (procedure),Y


In [55]:
import glob
for file in glob.glob("results/*ce.json"):
    with open(file) as f:
        result = json.load(f)['result']
    acc1 = sum([r['candidates'][0]['label'] for r in result])
    acc5 = sum([max([c['label'] for c in r['candidates'][:5]]) for r in result])
    print(f'{file}\t{acc1}\t{acc5}')

results\bb_binary_ce.json	4356	4736
results\bb_binary_sce.json	4377	4762
results\bb_linear_ce.json	4297	4692
results\bb_linear_sce.json	4273	4705
results\bb_log_ce.json	4361	4723
results\bb_log_sce.json	4342	4707
results\cb_binary_ce.json	4343	4729
results\cb_binary_sce.json	4374	4747
results\cb_linear_ce.json	4280	4654
results\cb_linear_sce.json	4261	4631
results\cb_log_ce.json	4345	4708
results\cb_log_sce.json	4323	4706


In [97]:
# [r['candidates'][:5]['label'] for r in result]
sum([max([c['label'] for c in r['candidates'][:5]]) for r in result])

4706

In [99]:
with open('results/bb_binary_ce.json') as f:
    result = json.load(f)['result']
N = len(result)
baseline_acc1 = sum([r['candidates'][0]['label'] for r in result])
baseline_acc5 = sum([max([c['label'] for c in r['candidates'][:5]]) for r in result])
print(N, baseline_acc1, baseline_acc5, baseline_acc5/N)

5302 4356 4736 0.8932478310071671


In [103]:
5302*.603

3197.1059999999998